In [0]:
from google.colab import auth, drive

auth.authenticate_user()

In [0]:
!earthengine authenticate

In [0]:
# Import the Earth Engine API and initialize it.
import ee
import time
import os
from pprint import pprint
import numpy
import tensorflow as tf
tf.enable_eager_execution()

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout

# Montamos la unidad de trabajo de nuestro drive
drive.mount('/content/drive/')

# Cambiamos la ruta de trabajo a nuestro drive
os.chdir('/content/drive/My Drive/')

ee.Initialize()

Funciones a utilizar

In [0]:
# Cloud masking function.
def maskL8sr(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('pixel_qa')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask).select(bands).divide(10000)

def parse_tfrecord(example_proto):
  """The parsing function.

  Read a serialized example into the structure defined by featuresDict.

  Args:
    example_proto: a serialized Example.
  
  Returns: 
    A tuple of the predictors dictionary and the label, cast to an `int32`.
  """
  parsed_features = tf.io.parse_single_example(example_proto, featuresDict)
  labels = parsed_features.pop(label)
  return parsed_features, tf.cast(labels, tf.int32)

def normalizedDifference(a, b):
  """Compute normalized difference of two inputs.

  Compute (a - b) / (a + b).  If the denomenator is zero, add a small delta.  

  Args:
    a: an input tensor with shape=[1]
    b: an input tensor with shape=[1]

  Returns:
    The normalized difference as a tensor.
  """
  nd = (a - b) / (a + b)
  nd_inf = (a - b) / (a + b + 0.000001)
  return tf.where(tf.math.is_finite(nd), nd, nd_inf)

def addNDVI(features, label):
  """Add NDVI to the dataset.
  Args: 
    features: a dictionary of input tensors keyed by feature name.
    label: the target label
  
  Returns:
    A tuple of the input dictionary with an NDVI tensor added and the label.
  """
  features['NDVI'] = normalizedDifference(features['B5'], features['B4'])
  return features, label

# Keras requires inputs as a tuple.  Note that the inputs must be in the
# right shape.  Also note that to use the categorical_crossentropy loss,
# the label needs to be turned into a one-hot vector.
def toTuple(dict, label):
  return tf.transpose(list(dict.values())), tf.one_hot(indices=label, depth=nClasses)

Ejemplo 1.1 - Preprocesar datos de Earth Engine y transformarlos a tfRecord.

In [0]:
# Use these bands for prediction.
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
label = 'landcover'

# Use Landsat 8 surface reflectance data.
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
# The image input data is a 2018 cloud-masked median composite.
image = l8sr.filterDate('2018-01-01', '2018-12-31').map(maskL8sr).median()

# Change the following two lines to use your own training data.
labels = ee.FeatureCollection('GOOGLE/EE/DEMOS/demo_landcover_labels')

# Sample the image at the points and add a random column.
sample = image.sampleRegions(collection=labels, properties=[label], scale=30).randomColumn()

# Partition the sample approximately 70-30.
training = sample.filter(ee.Filter.lt('random', 0.7))
testing = sample.filter(ee.Filter.gte('random', 0.7))

# Names for output files.
trainFilePrefix = 'Training_demo'
testFilePrefix = 'Testing_demo'
fileNameSuffix = '.tfrecord.gz'

# This is list of all the properties we want to export.
featureNames = list(bands)
featureNames.append(label)

trainingTask = ee.batch.Export.table.toDrive(
    collection=training,
    folder='agricultura/bands tfrecord',
    description='Training Export', 
    fileNamePrefix=trainFilePrefix, 
    fileFormat='TFRecord',
    selectors=featureNames)

testingTask = ee.batch.Export.table.toDrive(
    collection=testing,
    folder='agricultura/bands tfrecord',
    description='Testing Export', 
    fileNamePrefix=testFilePrefix, 
    fileFormat='TFRecord',
    selectors=featureNames)

# Start the tasks.
trainingTask.start()
testingTask.start()

# Poll the training task until it's done.
while trainingTask.active():
  print('Polling for task (id: {}).'.format(trainingTask.id))
  time.sleep(30)
print('Done with training export.')

# Poll the testing task until it's done.
while testingTask.active():
  print('Polling for task (id: {}).'.format(testingTask.id))
  time.sleep(30)
print('Done with testing export.')

Ejemplo 1.2 - Obtener el fichero tfRecord y prepararlo para tensorflow

In [0]:
# Get the tfRecord from Google drive
trainFilePath = 'agricultura/bands tfrecord' + trainFilePrefix + fileNameSuffix
testFilePath = 'agricultura/bands tfrecord' + testFilePrefix + fileNameSuffix

# Create a dataset from the TFRecord file in Cloud Storage.
trainDataset = tf.data.TFRecordDataset(trainFilePath, compression_type='GZIP')
testDataset = tf.data.TFRecordDataset(testFilePath, compression_type='GZIP')

# Print the records to check.
# print(iter(trainDataset).next())
# print(iter(testDataset).next())

# List of fixed-length features, all of which are float32.
columns = [tf.io.FixedLenFeature(shape=[1], dtype=tf.float32) for k in featureNames]

# Dictionary with names as keys, features as values.
featuresDict = dict(zip(featureNames, columns))

# Map the function over the dataset.
parsedDatasetTrain = trainDataset.map(parse_tfrecord, num_parallel_calls=5)
parsedDatasetTest = testDataset.map(parse_tfrecord, num_parallel_calls=5)

# Print the first parsed record to check.
# pprint(iter(parsedDatasetTrain).next())
# pprint(iter(parsedDatasetTest).next())

Ejemplo 1.3 - Crear el modelo y empezar a entrenar

In [133]:
# How many classes there are in the model.
nClasses = 3
labelsName= ['bare', 'vegetation', 'water']
batches = 10
nEpochs = 3
stepPerEpochs = 100

# Add NDVI.
inputTrainDataset = parsedDatasetTrain.map(addNDVI)
inputTestDataset = parsedDatasetTest.map(addNDVI)

# Repeat the input dataset as many times as necessary in batches of 10.
inputTrainDataset = inputTrainDataset.map(toTuple).repeat().batch(batches)
inputTestDataset = inputTestDataset.map(toTuple).batch(1)

# Define the layers in the model.
model = Sequential()
model.add(Dense(64, activation=tf.nn.relu))
model.add(Dropout(0.2))
model.add(Dense(nClasses, activation=tf.nn.softmax))

# Compile the model with the specified loss function.
model.compile(optimizer=tf.train.AdamOptimizer(),loss='categorical_crossentropy',metrics=['accuracy'])

# Fit the model to the training data.
# Don't forget to specify `steps_per_epoch` when calling `fit` on a dataset.
model.fit(x=inputTrainDataset.shuffle(50), epochs=nEpochs, steps_per_epoch=stepPerEpochs)
print("Training finished")

# Evaluate the model
model.evaluate(x=inputTestDataset, steps=100)
print("Evaluation finished")

Epoch 1/3
100/100 [==============================] - 2s 17ms/step - loss: 0.9138 - acc: 0.5603
Epoch 2/3
100/100 [==============================] - 1s 7ms/step - loss: 0.6236 - acc: 0.9038
Epoch 3/3
100/100 [==============================] - 1s 7ms/step - loss: 0.4162 - acc: 0.9533
Training finished
 23/100 [=====>........................] - ETA: 0s - loss: 0.4578 - acc: 0.9130 WARNING:tensorflow:Your dataset ran out of data; interrupting training. Make sure that your dataset can generate at least `steps * epochs` batches (in this case, 100 batches). You may need to use the repeat() function when building your dataset.
Evaluation finished


In [0]:
def NDVI(entrada):
    ndvi = entrada.addBands(entrada.normalizedDifference(['B8','B4']).rename('NDVI'))
    return (ndvi)

bands = ['B4','B8']
label = 'agriculture _activity'

parcelas = ee.FeatureCollection("ft:1DLwfbRr4i4fDh0i-Zy8mB59F5L_HOlqojE0JHDYn")
coleccion_ndvi = ee.ImageCollection('COPERNICUS/S2').filterBounds(parcelas).filterDate('2017‐09‐01','2018‐08‐31').filterMetadata('CLOUD_COVERAGE_ASSESSMENT','less_than',5).map(NDVI).select('NDVI')

image = coleccion_ndvi.median()

# Sample the image at the points and add a random column.
sample = image.sampleRegions(collection=parcelas, properties=[label], scale=30).randomColumn()

# Partition the sample approximately 70-30.
training = parcelas.filter(ee.Filter.lt('random', 0.7))
testing = parcelas.filter(ee.Filter.gte('random', 0.7))


from pprint import pprint

# Print the first couple points to verify.
pprint({'training': training.first().getInfo()})
pprint({'testing': testing.first().getInfo()})



{'training': None}
{'testing': None}
